In [ ]:
Project Title: Integrated Retail Analytics & Sales Prediction using Machine Learning

In [ ]:
Type: Supervised Machine Learning — Regression

In [ ]:
Contribution: Individual
By Suzi Sharma

In [ ]:
GitHub link: 

In [ ]:
# Project summary: This project — Integrated Retail Analytics for Store Optimization — constructs an end-to-end pipeline to transform sales, store 
# metadata, and auxiliary feature data into actionable business insights and a predictive model for store-level sales. The pipeline has three main 
# pillars: Exploratory Data Analysis (UBM), statistical validation of hypotheses, and predictive modeling with deployment readiness.
# First, we load and harmonize the three user-provided CSVs and normalize column names to avoid fragile indexing. We build a master dataframe by 
# merging sales with store metadata and features (on store and date when available). We create a consistent target column sales_amount (mapped from 
# weekly_sales in these input files) and generate time features (year, month, day, dayofweek, is_weekend).

# Exploratory Data Analysis strictly follows the UBM rule and produces 15+ charts:

# Univariate charts (distribution of sales, top stores, weekly_sales histogram, day-of-week counts, holiday distribution) reveal skewness in sales (heavy tail skew), concentration of records in top stores, and operational calendar patterns.

# Bivariate charts (weekly_sales vs dept averages, holiday boxplots, correlation matrix, month-vs-store heatmap) highlight departments with strongest average sales, the holiday uplift, and numeric relationships.

# Multivariate analyses (scatter-matrix, store-month grouped bars, time-series trend, bubble chart, dept×holiday interaction) expose seasonality, interactions (e.g., specific departments perform better on holidays), and trends useful for inventory planning.

# From EDA we derive three hypotheses and statistically test them (t-tests / Pearson correlation):

# Weekend mean sales > weekday mean sales (tested via t-test).

# Promotion days have higher sales (if promo exists; t-test
# Price and sales are negatively correlated (Pearson’s r, if price present).
# Cleaning & preprocessing use production-friendly strategies:
# Numeric missing values → median imputation
# Categorical missing → "Unknown"
# Outlier handling → IQR capping (clip)
# Categorical encoding → One-Hot Encoding in a ColumnTransformer
# Scaling → StandardScaler for numeric features
# All preprocessing is wrapped in scikit-learn Pipelines — this ensures reproducibility and easy deployment.
# Modeling compares three models: Linear Regression (baseline), Random Forest Regressor, and Gradient Boosting Regressor. We use K-Fold cross-
# validation to estimate generalization (neg_root_mean_squared_error for easier RMSE interpretation). For RandomForest and GradientBoosting we run 
# lightweight GridSearchCV to tune key hyperparameters. We evaluate on hold-out test set using RMSE, MAE and R². Model selection is RMSE-driven: 
# choose the model with smallest RMSE while also considering business interpretability and inference cost. Feature importance is extracted from the 
# tree-based model and saved.
# Deployment readiness: final model is saved as a scikit-learn Pipeline (preprocessing + estimator) using joblib — this binary can be loaded for
# inference in a Flask/FastAPI service. Plots, evaluation CSVs, hypothesis results and feature importance CSVs are saved under /mnt/data (or local 
# working dir) for reporting or automated dashboards.
# Business impact highlights:
# Seasonality & day-of-week effects inform staffing and inventory scheduling.
# Holiday/promotional uplift quantification justifies promotion investment and targeting.
# Top-store concentration suggests prioritizing high-return stores for new initiatives.
# Predictive sales model enables demand planning, improved stock replenishment, and better promotion ROI measurement.
# All logic is commented, packaged in reproducible steps, and saved artifacts make the notebook deployment-ready.

In [ ]:
# Build an end-to-end, deployment-ready retail analytics pipeline that ingests three CSVs (sales, stores, features), performs thorough Exploratory Data
# Analysis following the UBM rule (Univariate, Bivariate, Multivariate), tests three hypothesis statements statistically, engineers features, trains 
# and compares multiple ML models to predict store-level sales (weekly sales / sales_amount), tunes models with cross-validation & hyperparameter search,
# and exports artifacts (plots, model pipeline, evaluation) so the solution is ready to serve (Flask/FastAPI) or package. Emphasize production-grade 
# practices (pipelines, saving artifacts, clear comments, error handling).

In [ ]:
# Notebook: Integrated Retail Analytics for Store Optimization
# Requirements: pandas, numpy, matplotlib, scikit-learn, scipy, joblib
# Run everything in one cell in a Jupyter Notebook to be "deployment-ready".

import os, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import joblib
import json

# -------------------------
# CONFIG & PATHS
# -------------------------
SALES_CSV    = "sales data-set.csv"
STORES_CSV   = "stores data-set.csv"
FEATURES_CSV = "Features data set.csv"

OUT_DIR = "retail_artifacts"
PLOTS_DIR = os.path.join(OUT_DIR, "plots")
MODELS_DIR = os.path.join(OUT_DIR, "models")
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# -------------------------
# UTILS
# -------------------------
def safe_read_csv(path):
    """Try common encodings and return a DataFrame."""
    for enc in [None, "utf-8", "latin1", "cp1252"]:
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    raise FileNotFoundError(f"Unable to read {path}")

def normalize_columns(df):
    out = df.copy()
    out.columns = [c.strip().lower().replace(" ", "_") for c in out.columns]
    return out
def safe_merge_sales_features(sales_path, features_df, chunksize=50000):
    """
    Merge sales and features safely in chunks to avoid MemoryError.
    Handles multiple date formats automatically.
    """
    feats = features_df.copy()

    # robust date conversion for features
    feats["date"] = pd.to_datetime(
        feats["date"], errors="coerce", dayfirst=True, infer_datetime_format=True
    ).dt.date

    merged_chunks = []
    for chunk in pd.read_csv(sales_path, chunksize=chunksize):
        ch = normalize_columns(chunk)
        ch["date"] = pd.to_datetime(
            ch["date"], errors="coerce", dayfirst=True, infer_datetime_format=True
        ).dt.date
        merged = ch.merge(feats, how="left", on=["store", "date"])
        merged_chunks.append(merged)

    return pd.concat(merged_chunks, ignore_index=True)

# def safe_merge_sales_features(sales_path, features_df, chunksize=50000):
#     """
#     Merge sales and features safely in chunks to avoid MemoryError.
#     """
#     feats = features_df.copy()
#     feats["date"] = pd.to_datetime(feats["date"]).dt.date
#     merged_chunks = []
#     for chunk in pd.read_csv(sales_path, chunksize=chunksize):
#         ch = normalize_columns(chunk)
#         ch["date"] = pd.to_datetime(ch["date"]).dt.date
#         merged = ch.merge(feats, how="left", on=["store", "date"])
#         merged_chunks.append(merged)
#     return pd.concat(merged_chunks, ignore_index=True)

def save_fig(fig, fname):
    path = os.path.join(PLOTS_DIR, fname)
    fig.savefig(path, bbox_inches="tight")
    plt.close(fig)
    return path

# -------------------------
# LOAD DATA
# -------------------------
df_stores = normalize_columns(safe_read_csv(STORES_CSV))
df_features = normalize_columns(safe_read_csv(FEATURES_CSV))

# Keep only needed columns in features
feat_cols = ["store","date","temperature","fuel_price","cpi",
             "unemployment","isholiday"]
df_feat_small = df_features[[c for c in feat_cols if c in df_features.columns]]

# -------------------------
# MASTER MERGE  (CHUNKED)
# -------------------------
df_master = safe_merge_sales_features(SALES_CSV, df_feat_small, chunksize=50000)
df_master = df_master.merge(df_stores, on="store", how="left")

print("Merged shape:", df_master.shape)

# -------------------------
# TARGET: sales_amount
# -------------------------
for cand in ["sales_amount","weekly_sales","sales","unit_sales","amount","revenue"]:
    if cand in df_master.columns:
        df_master["sales_amount"] = df_master[cand].astype(float)
        break
if "sales_amount" not in df_master.columns:
    raise KeyError("No recognisable sales column found.")

# -------------------------
# BASIC EDA & CLEANING
# -------------------------
df_master["date"] = pd.to_datetime(df_master["date"], errors="coerce")
df_master["year"] = df_master["date"].dt.year
df_master["month"] = df_master["date"].dt.month
df_master["dayofweek"] = df_master["date"].dt.dayofweek
df_master["is_weekend"] = df_master["dayofweek"].isin([5,6]).astype(int)

# save summaries
df_master.describe(include="all").T.to_csv(os.path.join(OUT_DIR,"variables_description.csv"))
df_master.nunique(dropna=False).to_csv(os.path.join(OUT_DIR,"unique_counts.csv"))
df_master.isnull().sum().sort_values(ascending=False).to_csv(os.path.join(OUT_DIR,"missing_summary.csv"))

# simple imputation
num_cols = df_master.select_dtypes(include=[np.number]).columns
cat_cols = df_master.select_dtypes(include=["object","category"]).columns
df_prep = df_master.copy()
for c in num_cols:
    df_prep[c] = df_prep[c].fillna(df_prep[c].median())
for c in cat_cols:
    df_prep[c] = df_prep[c].fillna("Unknown")

# -------------------------
# UBM CHARTS  (memory-safe)
# -------------------------
# Chart 1: distribution of sales_amount
fig = plt.figure(figsize=(8,5))
df_prep["sales_amount"].hist(bins=60)
plt.title("Distribution: sales_amount")
save_fig(fig,"01_sales_amount_hist.png")

# Chart 2: top stores
fig = plt.figure(figsize=(10,5))
df_prep["store"].value_counts().nlargest(20).plot(kind="bar")
plt.title("Top20 stores by records")
save_fig(fig,"02_top20_stores.png")

# Chart 3: weekly_sales hist
if "weekly_sales" in df_prep.columns:
    fig = plt.figure(figsize=(8,5))
    df_prep["weekly_sales"].hist(bins=50)
    plt.title("weekly_sales distribution")
    save_fig(fig,"03_weekly_sales_hist.png")

# Chart 4: dayofweek counts
if "dayofweek" in df_prep.columns:
    fig = plt.figure(figsize=(7,4))
    df_prep["dayofweek"].value_counts().sort_index().plot(kind="bar")
    plt.title("Records by dayofweek")
    save_fig(fig,"04_dayofweek_bar.png")

# Chart 5: IsHoliday
if "isholiday" in df_prep.columns:
    fig = plt.figure(figsize=(6,4))
    df_prep["isholiday"].value_counts().plot(kind="bar")
    plt.title("IsHoliday distribution")
    save_fig(fig,"05_isholiday_bar.png")

# Chart 6: avg weekly_sales by dept
if "dept" in df_prep.columns:
    fig = plt.figure(figsize=(10,5))
    df_prep.groupby("dept")["weekly_sales"].mean().nlargest(20).plot(kind="bar")
    plt.title("Top20 depts avg weekly_sales")
    save_fig(fig,"06_avg_weekly_by_dept.png")

# Chart 7: weekly_sales by holiday (box)
if "isholiday" in df_prep.columns:
    groups=[df_prep[df_prep["isholiday"]==g]["weekly_sales"]
            for g in df_prep["isholiday"].unique()]
    fig = plt.figure(figsize=(6,5))
    plt.boxplot(groups, labels=[str(g) for g in df_prep["isholiday"].unique()])
    plt.title("weekly_sales by isholiday")
    save_fig(fig,"07_box_weekly_by_holiday.png")

# Chart 8: correlation (safe)
num_candidates=["sales_amount","weekly_sales","temperature",
                "fuel_price","cpi","unemployment","size"]
present=[c for c in num_candidates if c in df_prep.columns]
if present:
    corr=df_prep[present].sample(min(50000,len(df_prep)),random_state=42).corr()
    fig=plt.figure(figsize=(8,6))
    plt.imshow(corr,cmap="coolwarm")
    plt.xticks(range(len(corr)),corr.columns,rotation=90)
    plt.yticks(range(len(corr)),corr.columns)
    plt.colorbar()
    plt.title("Correlation (subset)")
    save_fig(fig,"08_corr_safe.png")

# Chart 9: month vs store heatmap
if "month" in df_prep.columns:
    top12=df_prep["store"].value_counts().nlargest(12).index
    pivot=df_prep[df_prep["store"].isin(top12)].pivot_table(
        index="month",columns="store",values="weekly_sales",aggfunc="mean")
    fig=plt.figure(figsize=(12,6))
    plt.imshow(pivot.fillna(0),aspect="auto")
    plt.colorbar(); plt.title("Month vs Store heatmap")
    save_fig(fig,"09_month_store_heatmap.png")

# Chart 10: scatter-matrix (safe)
sm_vars=[c for c in present if c!="sales_amount"][:4]+["sales_amount"]
if len(sm_vars)>=3:
    sm=scatter_matrix(df_prep[sm_vars].dropna().sample(min(3000,len(df_prep)),random_state=42),
                      figsize=(9,9),diagonal="hist")
    plt.suptitle("Scatter matrix")
    plt.savefig(os.path.join(PLOTS_DIR,"10_scatter_matrix.png")); plt.close()

# Chart 14: avg weekly_sales by dept & holiday
if {"dept","isholiday"}.issubset(df_prep.columns):
    pv=df_prep.pivot_table(index="dept",columns="isholiday",
                           values="weekly_sales",aggfunc="mean").fillna(0).head(20)
    pv.plot(kind="bar",figsize=(12,6)).get_figure().savefig(os.path.join(PLOTS_DIR,"14_dept_holiday_avg.png")); plt.close()

# Chart 15: top20 dept counts
if "dept" in df_prep.columns:
    fig=plt.figure(figsize=(10,5))
    df_prep["dept"].value_counts().head(20).plot(kind="bar")
    plt.title("Top20 dept counts")
    save_fig(fig,"15_top20_dept_counts.png")

# -------------------------
# HYPOTHESIS TESTING (3 statements)
# -------------------------
hypothesis_results = {}

# Hypothesis 1: weekend mean > weekday mean
if "is_weekend" in df_prep.columns and "sales_amount" in df_prep.columns:
    weekend = df_prep[df_prep["is_weekend"]==1]["sales_amount"].dropna()
    weekday = df_prep[df_prep["is_weekend"]==0]["sales_amount"].dropna()
    if len(weekend) > 10 and len(weekday) > 10:
        n = min(2000, len(weekend), len(weekday))
        tstat, pval = stats.ttest_ind(weekend.sample(n, random_state=42), weekday.sample(n, random_state=42), equal_var=False)
        hypothesis_results["weekend_vs_weekday"] = {"tstat":float(tstat), "pval":float(pval), "weekend_mean":float(weekend.mean()), "weekday_mean":float(weekday.mean())}

# Hypothesis 2: promo days > non-promo days (if promo exists)
if "promo" in df_prep.columns and "sales_amount" in df_prep.columns:
    p_yes = df_prep[df_prep["promo"]==1]["sales_amount"].dropna()
    p_no  = df_prep[df_prep["promo"]==0]["sales_amount"].dropna()
    if len(p_yes) > 10 and len(p_no) > 10:
        n = min(2000, len(p_yes), len(p_no))
        t2, p2 = stats.ttest_ind(p_yes.sample(n, random_state=42), p_no.sample(n, random_state=42), equal_var=False)
        hypothesis_results["promo_effect"] = {"tstat":float(t2), "pval":float(p2), "promo_mean":float(p_yes.mean()), "no_promo_mean":float(p_no.mean())}

# Hypothesis 3: price negatively correlated with sales (if price present)
if "price" in df_prep.columns and "sales_amount" in df_prep.columns:
    corr = df_prep[["price","sales_amount"]].dropna().corr().iloc[0,1]
    hypothesis_results["price_sales_corr"] = {"pearson_r":float(corr)}

# Save hypothesis results
with open(os.path.join(OUT_DIR,"hypothesis_results.json"), "w") as f:
    json.dump(hypothesis_results, f, indent=2)

# -------------------------
# PREPROCESS & FEATURE SELECTION FOR ML (operate on df_prep)
# -------------------------
if "sales_amount" not in df_prep.columns:
    raise KeyError("sales_amount not found for ML target.")
y = df_prep["sales_amount"].copy()
X = df_prep.drop(columns=["sales_amount","date"], errors="ignore").copy()

# Drop very high-cardinality text fields to avoid exploding OHE
high_card = [c for c in X.columns if X[c].dtype==object and X[c].nunique()>200]
X = X.drop(columns=high_card, errors="ignore")

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define transformers (use columns from the reduced X)
num_features = [c for c in X.select_dtypes(include=[np.number]).columns.tolist()]
cat_features = [c for c in X.select_dtypes(include=["object","category"]).columns.tolist()]

num_pipeline = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
cat_pipeline = Pipeline([("imputer", SimpleImputer(strategy="constant", fill_value="Unknown")), ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

preprocessor = ColumnTransformer([("num", num_pipeline, num_features), ("cat", cat_pipeline, cat_features)])

# -------------------------
# MODELS: baseline + RF + GB (with light GridSearch)
# -------------------------
models = {
    "LinearRegression": Pipeline([("preproc", preprocessor), ("model", LinearRegression())]),
    "RandomForest": Pipeline([("preproc", preprocessor), ("model", RandomForestRegressor(random_state=42, n_jobs=-1))]),
    "GradientBoosting": Pipeline([("preproc", preprocessor), ("model", GradientBoostingRegressor(random_state=42))])
}

# Cross-val baseline
cv = KFold(n_splits=3, shuffle=True, random_state=42)
cv_results = {}
for name, pipe in models.items():
    try:
        scores = cross_val_score(pipe, X_train, y_train, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)
        cv_results[name] = {"rmse_mean": -float(np.mean(scores)), "rmse_std": float(np.std(scores))}
    except Exception as e:
        cv_results[name] = {"error": str(e)}
with open(os.path.join(MODELS_DIR,"cv_results.json"), "w") as f:
    json.dump(cv_results, f, indent=2)

# GridSearch for RandomForest and GradientBoosting (wrapped in try/except to avoid runaway compute)
param_grid_rf = {"model__n_estimators":[50,100],"model__max_depth":[10,20,None]}
param_grid_gb = {"model__n_estimators":[100,200],"model__learning_rate":[0.05,0.1],"model__max_depth":[3,5]}

gs_rf = GridSearchCV(models["RandomForest"], param_grid_rf, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)
gs_gb = GridSearchCV(models["GradientBoosting"], param_grid_gb, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)

# Fit carefully with try/except in case resources are limited
try:
    gs_rf.fit(X_train, y_train)
    best_rf = gs_rf.best_estimator_
except Exception as e:
    print("RandomForest GridSearch failed or was stopped:", e)
    # fallback: fit a default RandomForest pipeline
    best_rf = models["RandomForest"]
    best_rf.fit(X_train, y_train)

try:
    gs_gb.fit(X_train, y_train)
    best_gb = gs_gb.best_estimator_
except Exception as e:
    print("GradientBoosting GridSearch failed or was stopped:", e)
    best_gb = models["GradientBoosting"]
    best_gb.fit(X_train, y_train)

# Evaluate on test set
evals = {}
for name, est in [("LinearRegression", models["LinearRegression"]),
                  ("RandomForest", best_rf),
                  ("GradientBoosting", best_gb)]:
    est.fit(X_train, y_train)
    preds = est.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    rmse = mse ** 0.5
    mae = mean_absolute_error(y_test, preds)
    r2  = r2_score(y_test, preds)
    evals[name] = {"rmse": float(rmse), "mae": float(mae), "r2": float(r2)}
    
    
with open(os.path.join(MODELS_DIR,"evaluation_results.json"), "w") as f:
    json.dump(evals, f, indent=2)

# Save model comparison plot (if numeric RMSE available)
names = list(evals.keys())
rmses = [evals[n].get("rmse", np.nan) for n in names]
fig = plt.figure(figsize=(8,5)); plt.bar(names, rmses); plt.title("Model RMSE (test)"); plt.ylabel("RMSE"); plt.savefig(os.path.join(PLOTS_DIR,"model_rmse_comparison.png")); plt.close()

# Feature importances (from RandomForest) - only if fitted
try:
    rf_pipeline = best_rf
    # we attempt to extract feature names only if OHE exists and has get_feature_names_out
    if hasattr(rf_pipeline.named_steps["preproc"].transformers_[1][1], "named_steps"):
        ohe = rf_pipeline.named_steps["preproc"].transformers_[1][1].named_steps.get("ohe", None)
    else:
        ohe = None
    ohe_feature_names = []
    if ohe is not None and hasattr(ohe, "get_feature_names_out"):
        ohe_feature_names = list(ohe.get_feature_names_out(cat_features))
    feature_names = num_features + ohe_feature_names
    rf_model = rf_pipeline.named_steps["model"]
    if hasattr(rf_model, "feature_importances_"):
        importances = rf_model.feature_importances_
        fi = pd.Series(importances, index=feature_names).sort_values(ascending=False)
        fi.head(30).to_csv(os.path.join(MODELS_DIR,"rf_feature_importance.csv"))
except Exception as e:
    print("Could not extract RF feature importances:", e)

# Save best model (choose by rmse if present, else fallback)
best_name = None
try:
    best_name = min({k:v for k,v in evals.items() if "rmse" in v}, key=lambda k: evals[k]["rmse"])
except Exception:
    best_name = "RandomForest"
best_map = {"LinearRegression":models["LinearRegression"], "RandomForest":best_rf, "GradientBoosting":best_gb}
final_model = best_map.get(best_name, best_rf)
joblib.dump(final_model, os.path.join(MODELS_DIR,"final_model_pipeline.pkl"))

# -------------------------
# REPORT & ARTIFACTS
# -------------------------
report = {
    "problem_statement": "Predict sales_amount (weekly sales) for store-level demand forecasting to improve inventory, promotions, and staffing.",
    "cv_results": cv_results,
    "gridsearch_rf_best": getattr(gs_rf, "best_params_", {}),
    "gridsearch_gb_best": getattr(gs_gb, "best_params_", {}),
    "eval_results": evals,
    "best_model": best_name
}
with open(os.path.join(OUT_DIR,"project_report.json"), "w") as f:
    json.dump(report, f, indent=2)

print("Artifacts saved under:", OUT_DIR)
print("Best model:", best_name)


Merged shape: (421570, 12)


In [9]:
print(df_sales["date"].head())
print(df_features["date"].head())


0    2010-05-02
1    2010-12-02
2           NaT
3           NaT
4    2010-05-03
Name: date, dtype: object
0    05/02/2010
1    12/02/2010
2    19/02/2010
3    26/02/2010
4    05/03/2010
Name: date, dtype: object


In [18]:
import joblib
import pandas as pd

# 👉 Load the trained pipeline
model = joblib.load("retail_artifacts/models/final_model_pipeline.pkl")

# 👉 See the columns the model expects
print("Expected columns:\n", model.feature_names_in_)


Expected columns:
 ['store' 'dept' 'weekly_sales' 'isholiday_x' 'temperature' 'fuel_price'
 'cpi' 'unemployment' 'isholiday_y' 'type' 'size' 'year' 'month'
 'dayofweek' 'is_weekend']
